# 1. RAG chain 구현 구문

In [4]:
# RAG chain 설계 및 LLM 연동을 위한 모듈
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.prompts import ChatPromptTemplate, ChatMessagePromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough, RunnableWithMessageHistory
from langchain_community.tools import TavilySearchResults
from langchain_core.documents import Document

# 평가 알로리즘 모듈
from langchain_core.output_parsers import JsonOutputParser,StrOutputParser
from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

from ragas import EvaluationDataset, RunConfig, evaluate
from ragas.metrics import LLMContextRecall, Faithfulness, LLMContextPrecisionWithReference, AnswerRelevancy

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

from pydantic import BaseModel, Field


# 메모리 관련 모듈
from langchain_core.chat_history import InMemoryChatMessageHistory

from langchain.chains import RetrievalQA
from langchain.schema import AIMessage, HumanMessage




from textwrap import dedent
from operator import itemgetter

from dotenv import load_dotenv
load_dotenv()


True

In [ ]:
########################################################
# config 목록
########################################################
COLLECTION_NAME = "bluer_db_openai"
PERSIST_DIRECTORY = "vector_store/chroma/bluer_db"
EMBEDDING_MODEL_NAME = "text-embedding-3-small"
embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)
MODEL_NAME = 'gpt-4o-mini'


########################################################
# vector_db에서 데이터 불러오기
########################################################

# vector store 연결
vector_store = Chroma(
    embedding_function=EMBEDDING_MODEL,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)

# 저장된 데이터 내용 확인
documents = vector_store._collection.get()['documents']
metadatas = vector_store._collection.get()['metadatas']

print(f"Documents: {documents[:5]}") 
print(f"Metadatas: {metadatas[:5]}")

Documents: ['foodDetailTypes: 스시\nheaderInfo_nameKR: 스시조\nheaderInfo_nameEN: Sushi Cho\nheaderInfo_nameCN: \nheaderInfo_bookYear: 2025\nheaderInfo_ribbonType: 3\ndefaultInfo_chefName: \ndefaultInfo_phone: 02-317-0373\ndefaultInfo_openHours: \ndefaultInfo_closeHours: \ndefaultInfo_openHoursWeekend: \ndefaultInfo_closeHoursWeekend: \ndefaultInfo_dayOff: 연중무휴\ndefaultInfo_app2Yn: False\nstatusInfo_parking: 가능\nstatusInfo_creditCard: y\nstatusInfo_visit: 웨스틴조선호텔 20층\nstatusInfo_menu: 런치(Hall)(1인 15만5천원~20만5천원), 디너(Hall)(1인 19만4천원~33만원), 스시조회덮밥(10만원, 프리미엄 13만원), 복가라아게돌솥밥(12만원), 굴돌솥밥(7만8천원), 활새우튀김(11만원), 조리장특선모둠스시(14만5천원)\nstatusInfo_priceRange: 25만원 이상\nstatusInfo_openDate: 2008년\nstatusInfo_businessHours: 12:00~15:00/17:30~22:00(마지막 주문 21:30)\njuso_detailAddress: 웨스틴조선호텔 20층\njuso_roadAddrPart1: 서울특별시 중구 소공로 106\njuso_engAddr: 106, Sogong-ro, Jung-gu, Seoul\njuso_bdNm: 서울 웨스틴조선호텔\njuso_siNm: 서울특별시\njuso_sggNm: 중구\njuso_emdNm: 소공동\njuso_liNm: \njuso_rn: 소공로\njuso_buldMnnm: 106\njuso_buldSln

In [7]:
QUERY = "프라이빗 룸이 있는 한식집 추천해줘"  


# embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)


vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)


# GPT Model 생성
model = ChatOpenAI(
    model=MODEL_NAME,
    temperature=0 
)



# Retriever 생성
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}  # Number of top documents to retrieve
)



# Prompt Template 생성
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 한국의 블루리본 서베이 전문가입니다. 질문에 자세히 답해주세요."),
    ("human", "{question}")
])


#########################################
# Chain 생성
#########################################

retrieval_qa = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    return_source_documents=True,  # Include source documents in response
)


response = retrieval_qa({"query": QUERY})

print("응답:", response["result"])

응답: 프라이빗 룸이 있는 한식집으로는 김범준 셰프의 모던한식 레스토랑이 있습니다. 이곳은 제철 재료를 사용한 한식 기반의 요리를 제공하며, 프라이빗 룸에서 다이닝을 즐길 수 있는 분위기가 좋습니다. 식사로는 솥밥이 나오며, 두 개의 룸에서 각각 원테이블로 코스가 진행됩니다. 

주소: 서울특별시 강남구 압구정로75길 5, 리아빌딩 지하2층  
전화: 010-8859-5280  
영업시간: 12:00~16:00 / 18:00~22:00  
휴무일: 연중무휴  
메뉴: 한식맡김차림(15만원)


In [ ]:
QUERY = "프라이빗 룸이 있는 한식집 추천해줘"  

#########################################################
# InMemoryVectorStore 생성
#########################################################

store = {} 
# key : session_id, value : InMemoryChatMessageHistory ( session id별로 저장하는 기능이 없다.)

def get_session_history(session_id):
    '''
    ChatMessageHistory 객체를 반환하는 함수
    store에서 session_id의 History객체를 찾아서 반환, 없으면 생성해서 store 저장
    '''     
    
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    
    return store[session_id]



runnable = prompt_template | model

# Chain + ChatMessageHJistory => 대화 + 메세지 저장관리
chain = RunnableWithMessageHistory(
    runnable= runnable, # chain 객체(RunnableSequence)
    get_session_history=get_session_history, # session_id의 ChatMessageHistory객체를 반환하는 함수.
    input_messages_key= 'query',    # prompt_template에 입력 내용을 넣을 변수명.
    history_messages_key='history'  # prompt_template에 대화내역을 넣어줄 변수명.
)

########################################
# 질문을 Embedding Vector로 변환
########################################


embedding_query = embedding_model.embed_query(QUERY) # 한문장 변환.
print(type(embedding_query), len(embedding_query))


############################################################
# retriever
############################################################


# Retriever 생성 - "Map Reduce" 방식
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k":5, "fetch_k":10, "lambda_mult":0.5}
)


map_doc_prompt = ChatPromptTemplate.from_messages([
    ("system",  """
Use the following portion of a long document to see if any of the text is relevant to answer the question. 
Return any relevant text. If there is no relevant text, return : ''
-------
{context}
"""),
    ("human", "{question}"),
])



# 질문 - 문서 관련성을 비교하는 체인
map_doc_chain = map_doc_prompt | model

## retriever로 문서 조회 -> map_doc_chain으로 관련문서를 찾기 
def map_doc(inputs):
    """
    Runnable로 정의할 함수. 
    retriever가 조회한 문서들과 question을 받아서 map_doc_chain을 이용해 관련성을 확인한다.
    관련된 문서 내용만 모아서 반환.
    parameter
        inputs: dict[documents: list[Document], question:질문]. {"documents":retriever, "question":RunnablePassthrough()}
    """
    docs = inputs["documents"]   # list[Document, Document, Document, ...]
    question= inputs["question"] # str
    context = "" # 질문과 관련된 내용들만 모아 놓을 변수.
    for doc in docs:
        # Document와 question을 map_doc_chain에 전달해서 관련된 내용인지 확인.
        res = map_doc_chain.invoke({"context":doc.page_content, "question":question})
        context += res.content+"\n\n" # AIMessage.content

    return context

map_reduce_chain = {"documents":retriever, "question":RunnablePassthrough()} | RunnableLambda(map_doc)

<class 'list'> 1536


In [16]:
documents = retriever.get_relevant_documents("리본 두개 이상인 서울 한식집을 알려주세요.")
print(f"Retrieved {len(documents)} documents")
for doc in documents:
    print(doc.page_content)

Retrieved 5 documents
id: 6364
createdDate: 1304147828000
bookStatus: APPROVAL
chefName: 박승재
closeHours: nan
closeHoursWeekend: nan
dayOff: 일, 월, 화요일 휴무
openHours: nan
openHoursWeekend: nan
phone: 010-5538-3973
website: nan
gps_latitude: 37.552014
gps_longitude: 126.927861
ribbonType: nan
juso_detailAddress: nan
juso_roadAddrPart1: 서울특별시 마포구 와우산로30길 80
review_text: 한식 주점의 정석과도 같은 곳. 손맛 좋은 오너 겸 셰프가 내는 한식을 안주로 삼아 저녁때 술 한잔 즐기기 좋다. 소갈비찜과 스팸골뱅이는 빼놓지 말고 주문해야 하는 메뉴. 그날의 신선한 재료로 만드는 요리도 매번 기대하게 하는 메뉴다.
review_simple: 한식 주점의 정석과도 같은 곳. 손맛 좋은 오너 겸 셰프가 내는 한식을 안주로 삼아 저녁때 술 한잔 즐기기 좋다. 소갈비찜과 스팸골뱅이는 빼놓지 말고 주문해야 하는 메뉴. 그날의 신선한 재료로 만드는 요리도 매번 기대하게 하는 메뉴다.
statusInfo_businessHours: 18:00~23:00(마지막 주문 22:00) 
statusInfo_creditCard: y
statusInfo_menu: 미로소갈비찜(3만2천원), 닭튀김(2만원), 양념돼지목살구이(2만5천원), 골뱅이무침과스팸구이(2만2천원), 해물부추전, 애호박감자채전(각 1만8천원)
foodType_1: 한식(일반한식)
foodDetailType_1: 모던한식
foodType_2: 모던한식
foodDetailType_2: 한식주점
foodType_3: 한식주점
foodDetailType_3: nan
foodType_4: nan
foodDetailType_4: nan
foodType_5: 

In [17]:
for doc in docs:
    print("Context:", doc.page_content)
    print("Question:", question)

NameError: name 'docs' is not defined

#### Map_reduce 확인

In [14]:
r = map_reduce_chain.invoke("리본 두개 이상인 서울 한식집을 알려주세요.")
print(r)

''

''

''

''

''




#### 최종 답변 

In [ ]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

chain = ({"context":map_reduce_chain, "question":RunnablePassthrough()} 
        | final_prompt
        | model
        | StrOutputParser())